In [1]:
# Importing required libraries
import pandas as pd
import ast
import re

from helper_functions.helpers import clean, structure_conversation_spacy, structure_conversation, Parse_spacy, Parse
from resolution_helpers.object_phrase_resolution import extract_head_verb_object
from resolution_helpers.append_resolved_phrase import append_resolved_vp
from resolution_helpers.append_missing_aux import find_missing_aux_verb
from detect_and_resolve_vpe import detect_resolve_vpe_conversation, detect_resolve_vpe_conversation_spacy
from compare_with_annotations import compare_head_verb, compare_ellided_obj_parent

import json

text = []
with open("./Test_Data/new_convo_test_cases.txt") as file:
    text = [line.rstrip() for line in file]

# Removing empty lines from the list
for line in text:
    if(line == '' or line == '\n'):
        text.remove(line)

# Counting lines and number of cases where ellipsis is detected and where it isn't detected
cases_ellipsis = 0
cases_no_ellipsis = 0

error_conversations = []

detect_resolve_output_file = open("./Outputs/detect_resolve_output_annotated.txt", "w+")

corr_count = 0
ann_count = 0

conversation_final_vpe_list = []

2023-04-30 18:22:30.318916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-30 18:22:30.318973: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining 

In [2]:
# Running it for all sentences in our list
for index, conversation in enumerate(text):
   
    print(str(index) + ") ", end = " ", file = detect_resolve_output_file)
    print(conversation, end = "\n\n", file = detect_resolve_output_file)

    temp_entry_output_storage_file = open("./Outputs/TempConversationalOutput/new_convo_entry_" + str(index) + ".txt", "w")

    print("Entry - %d\n" %(index), file = temp_entry_output_storage_file)
    print("Scoring in STANFORD\n", file = temp_entry_output_storage_file)
    print("Scoring in STANFORD\n", file = detect_resolve_output_file)
    vpe_conversation_stanford = detect_resolve_vpe_conversation(clean(conversation), detect_resolve_output_file, temp_entry_output_storage_file)
    
    print("Scoring in SPACY\n", file = temp_entry_output_storage_file)
    print("Scoring in SPACY\n", file = detect_resolve_output_file)
    vpe_conversation_spacy = detect_resolve_vpe_conversation_spacy(clean(conversation), detect_resolve_output_file, temp_entry_output_storage_file)
    
    detected_vpe_cases_stanford = []
    for output_arr in vpe_conversation_stanford:
        if output_arr != []:
            detected_vpe_cases_stanford.append(output_arr)
            
    detected_vpe_cases_spacy = []
    for output_arr in vpe_conversation_spacy:
        if output_arr != []:
            detected_vpe_cases_spacy.append(output_arr)
    
    # Taking only those detections where both the parsers detected VPE in the conversation
    final_vpe_list = []
    
    for vpe_dict_stanford in detected_vpe_cases_stanford:
        for vpe_dict_spacy in detected_vpe_cases_spacy:
            if vpe_dict_stanford['category'] == vpe_dict_spacy['category'] and vpe_dict_stanford['site'] == vpe_dict_spacy['site'] and vpe_dict_stanford['sen_index'] == vpe_dict_spacy['sen_index']:
                
                # If Main verb is resolved and both Stanford Spacy head verb resolutions match then choose any since they both are the same
                if vpe_dict_stanford['resolved_head_verb'] is not None and vpe_dict_spacy['resolved_head_verb'] is not None and vpe_dict_stanford['resolved_head_verb'] == vpe_dict_spacy['resolved_head_verb']:
                    final_vpe_list.append(vpe_dict_stanford)
                # If Main verb is resolved and Stanford Spacy head verb resolutions don't match then giving preference to Spacy
                if vpe_dict_stanford['resolved_head_verb'] is not None and vpe_dict_spacy['resolved_head_verb'] is not None and vpe_dict_stanford['resolved_head_verb'] != vpe_dict_spacy['resolved_head_verb']:
                    final_vpe_list.append(vpe_dict_spacy)
                # If Main verb is resolved for Spacy but not for Stanford then choose any since they Spacy
                if vpe_dict_stanford['resolved_head_verb'] is None and vpe_dict_spacy['resolved_head_verb'] is not None:
                    final_vpe_list.append(vpe_dict_spacy)
                # If Main verb is resolved for Stanford but not for Spacy then choose any since they Stanford
                if vpe_dict_stanford['resolved_head_verb'] is not None and vpe_dict_spacy['resolved_head_verb'] is None:
                    final_vpe_list.append(vpe_dict_stanford)
                # If Main verb is not resolved for neither of them it implies Only the Object child is ellided and if the Ellided Object child's parents matches then choose any since they are the same. 
                # Note that matched Ellided Object Child's parent could also be None which would imply we couldn't resolve the ellipsis successfully
                if vpe_dict_stanford['resolved_head_verb'] is None and vpe_dict_spacy['resolved_head_verb'] is None and vpe_dict_stanford['ellided_obj_child_parent_verb'] == vpe_dict_spacy['ellided_obj_child_parent_verb']:
                    final_vpe_list.append(vpe_dict_spacy)
                # If Main verb is not resolved for neither of them it implies Only the Object child is ellided and if the Ellided Object child's parents don't match then choose Spacy
                if vpe_dict_stanford['resolved_head_verb'] is None and vpe_dict_spacy['resolved_head_verb'] is None and vpe_dict_stanford['ellided_obj_child_parent_verb'] != vpe_dict_spacy['ellided_obj_child_parent_verb']:
                    final_vpe_list.append(vpe_dict_spacy)
                
    final_vpe_list = extract_head_verb_object(final_vpe_list, structure_conversation_spacy(clean(conversation)))
    final_vpe_list = find_missing_aux_verb(final_vpe_list, structure_conversation_spacy(clean(conversation)))
    
    conversation_final_vpe_list.append(final_vpe_list)
    
    print("\nFINAL VPE\n", file = temp_entry_output_storage_file)
    print(final_vpe_list, file = temp_entry_output_storage_file)
    print("\nFINAL VPE\n", file = detect_resolve_output_file)
    print(final_vpe_list, file = detect_resolve_output_file)
    
    print("================================= \n\n", file = temp_entry_output_storage_file)
    
    print("================================= \n\n", file = detect_resolve_output_file)
    
    temp_entry_output_storage_file.close()
    temp_entry_read_file = open("./Outputs/TempConversationalOutput/new_convo_entry_" + str(index) + ".txt", "r")
    erroneous_entry = temp_entry_read_file.read()
    print(erroneous_entry)
    temp_entry_read_file.close()
    
detect_resolve_output_file.close()

Entry - 0

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0]
{3: 0}
[{1: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{}]


AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{}]


AFTER AUX BASED SCORING
[0]
{}
[{}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{}]


AFTER ADDING SCORE MULTIPLIERS
[0]
{}
[{}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[0]
{3: 0}
[{1: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{}]


AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{}]


AFTER AUX BASED SCORING
[0]
{}
[{}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{}]


AFTER ADDING SCORE MULTIPLIERS
[0]
{}
[{}]
-----------------------------



FINAL VPE

[{'category': 1, 'site': 3, 'licensor': 'is', 'lemmatized_licensor': 'be', 'sen_index': 1, 'resolved_head_verb': None, 'ellided_obj_child_parent_verb': 1, 'sen_type': 2, 'vpe_sent': ' Yes , it is.', 'resolved_object': 'a fine day', 'resolved_verb_p

Entry - 4

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0, 1, 2]
{3: 0}
[{1: 0, 3: 0, 5: 0}, {}, {2: 0}]


AFTER VERB LIST FILTERING
[0, 1, 2]
{}
[{1: 0, 3: 0, 5: 0}, {}, {2: 0}]


Prev 0 - 3 NUMBER + 1
Prev 0 - 3 FP SP SWITCH + 1
Prev 0 - 5 NUMBER + 1
Prev 0 - 5 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[0, 1, 2]
{}
[{1: 0, 3: 2, 5: 2}, {}, {2: 0}]


AFTER COMP CLAUSE BASED SCORING
[0, 1, 2]
{}
[{1: 0, 3: 2, 5: 2}, {}, {2: 0}]


AFTER AUX BASED SCORING
[0, 1, 2]
{}
[{1: 0, 3: 2, 5: 2}, {}, {2: 0}]


AFTER BACKTRACKING BASED SCORING
[0, 1, 2]
{}
[{1: 0, 3: 2, 5: 2}, {}, {2: 0}]


AFTER ADDING SCORE MULTIPLIERS
[0, 1, 2]
{}
[{1: 0.0, 3: 1.3, 5: 1.3}, {}, {2: 0}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[0, 1, 2]
{}
[{1: 0, 3: 0, 5: 0}, {}, {2: 0}]


AFTER VERB LIST FILTERING
[0, 1, 2]
{}
[{1: 0, 3: 0, 5: 0}, {}, {2: 0}]


Prev 0 - 1 IMPERATIVE SUBJECT AND MODAL ELLIPSIS RESPONSE + 1
Prev 0 - 3 NUMBER + 1
Prev 0 - 3 FP SP SWITCH + 1
Prev

Entry - 10

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0, 1, 2]
{1: 0}
[{3: 0}, {1: 0, 4: 0}, {2: 0}]


AFTER VERB LIST FILTERING
[0, 1, 2]
{1: 0}
[{3: 0}, {1: 0, 4: 0}, {2: 0}]


1 SAME SUBJECT EXACT MATCH + 2
Prev 1 - 1 IMPERATIVE SUBJECT AND MODAL ELLIPSIS RESPONSE + 1
Prev 1 - 4 SAME SUBJECT FP SP NO SWITCH + 2
AFTER NOMINAL SUBJECT BASED SCORING
[0, 1, 2]
{1: 2}
[{3: 0}, {1: 1, 4: 2}, {2: 0}]


AFTER COMP CLAUSE BASED SCORING
[0, 1, 2]
{1: -1}
[{3: 0}, {1: 1, 4: 2}, {2: 0}]


AFTER AUX BASED SCORING
[0, 1, 2]
{1: -1}
[{3: 0}, {1: 1, 4: 2}, {2: 0}]


AFTER BACKTRACKING BASED SCORING
[0, 1, 2]
{1: 0}
[{3: 0}, {1: 1, 4: 2}, {2: 0}]


AFTER ADDING SCORE MULTIPLIERS
[0, 1, 2]
{1: 0}
[{3: 0.0}, {1: 0.8, 4: 1.6}, {2: 0}]
-----------------------------


BEFORE VERB LIST FILTERING
[3, 4, 5]
{3: 0}
[{1: 0}, {0: 0, 3: 0, 5: 0}, {}]


AFTER VERB LIST FILTERING
[3, 4, 5]
{}
[{1: 0}, {0: 0, 5: 0}, {}]


Prev 3 - 1 NUMBER + 1
Prev 4 - 5 NUMBER + 1
Prev 4 - 5 TP - 1
AFTER NOMINAL SUBJECT BA

Entry - 14

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[2, 3, 4]
{0: 0}
[{3: 0}, {0: 0, 6: 0}, {1: 0}]


AFTER VERB LIST FILTERING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER ADDING SCORE MULTIPLIERS
[2, 3, 4]
{}
[{}, {}, {}]
-----------------------------


BEFORE VERB LIST FILTERING
[7, 8, 9]
{3: 0}
[{1: 0}, {1: 0, 3: 0}, {0: 0, 2: 0, 4: 0}]


AFTER VERB LIST FILTERING
[7, 8, 9]
{}
[{}, {1: 0, 3: 0}, {2: 0, 4: 0}]


Prev 8 - 1 SAME SUBJECT FP SP NO SWITCH + 2
Prev 8 - 3 SAME SUBJECT FP SP NO SWITCH + 2
Prev 9 - 2 NUMBER + 1
Prev 9 - 2 FP SP SWITCH + 1
Prev 9 - 4 NUMBER + 1
Prev 9 - 4 TP - 1
AFTER NOMINAL SUBJECT BASED SCORING
[7, 8, 9]
{}
[{}, {1: 2, 3: 2}, {2: 2, 4: 0}]


AFTER COMP CLAUSE BASED SCORING
[7, 8, 9]
{}
[{}, {1: 2, 3: 1}, {2: 2, 4: -1}]


AFTE

Entry - 17

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[19, 20, 21]
{3: 0, 6: 0}
[{1: 0}, {3: 0}, {}]


AFTER VERB LIST FILTERING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 0}, {}]


Prev 20 - 3 NUMBER + 1
AFTER NOMINAL SUBJECT BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 1}, {}]


AFTER COMP CLAUSE BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 1}, {}]


AFTER AUX BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 1}, {}]


AFTER BACKTRACKING BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 1}, {}]


AFTER ADDING SCORE MULTIPLIERS
[19, 20, 21]
{3: 0}
[{1: 0.0}, {3: 0.8}, {}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[19, 20, 21]
{3: 0, 6: 0}
[{1: 0}, {3: 0}, {}]


AFTER VERB LIST FILTERING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 0}, {}]


Prev 20 - 3 NUMBER + 1
AFTER NOMINAL SUBJECT BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 1}, {}]


AFTER COMP CLAUSE BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {3: 1}, {}]


AFTER AUX BASED SCORING
[19, 20, 21]
{3: 0}
[{1: 0}, {

...

Entry - 32

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0]
{}
[{2: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{2: 0}]


Prev 0 - 2 NUMBER + 1
Prev 0 - 2 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{2: 2}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{2: 2}]


AFTER AUX BASED SCORING
[0]
{}
[{2: 3}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{2: 3}]


AFTER ADDING SCORE MULTIPLIERS
[0]
{}
[{2: 3}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[0]
{}
[{2: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{2: 0}]


Prev 0 - 2 NUMBER + 1
Prev 0 - 2 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{2: 2}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{2: 2}]


AFTER AUX BASED SCORING
[0]
{}
[{2: 3}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{2: 3}]


AFTER ADDING SCORE MULTIPLIERS
[0]
{}
[{2: 3}]
-----------------------------



FINAL VPE

[{'category': 4, 'site': 3, 'licensor': 'can', 'lemmatized_licensor': 'can', 'sen_index': 1, 'resolved_head

Entry - 36

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[1, 2, 3]
{2: 0}
[{}, {1: 0, 3: 0}, {2: 0, 6: 0, 10: 0}]


AFTER VERB LIST FILTERING
[1, 2, 3]
{}
[{}, {3: 0}, {}]


Prev 2 - 3 SAME SUBJECT FP SP NO SWITCH + 2
AFTER NOMINAL SUBJECT BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER COMP CLAUSE BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER AUX BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER BACKTRACKING BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER ADDING SCORE MULTIPLIERS
[1, 2, 3]
{}
[{}, {3: 1.6}, {}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[1, 2, 3]
{2: 0}
[{}, {1: 0, 3: 0}, {2: 0, 6: 0, 10: 0}]


AFTER VERB LIST FILTERING
[1, 2, 3]
{}
[{}, {3: 0}, {}]


Prev 2 - 3 SAME SUBJECT FP SP NO SWITCH + 2
AFTER NOMINAL SUBJECT BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER COMP CLAUSE BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER AUX BASED SCORING
[1, 2, 3]
{}
[{}, {3: 2}, {}]


AFTER BACKTRACKING BASED SCORING
[1, 

Entry - 41

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[10, 11, 12]
{}
[{1: 0, 4: 0}, {2: 0}, {2: 0, 7: 0, 14: 0}]


AFTER VERB LIST FILTERING
[10, 11, 12]
{}
[{1: 0, 4: 0}, {2: 0}, {7: 0, 14: 0}]


Prev 10 - 4 IMPERATIVE SUBJECT AND MODAL ELLIPSIS RESPONSE + 1
Prev 11 - 2 SAME SUBJECT FP SP NO SWITCH + 2
Prev 12 - 7 NUMBER + 1
Prev 12 - 7 FP SP SWITCH + 1
Prev 12 - 14 IMPERATIVE SUBJECT AND MODAL ELLIPSIS RESPONSE + 1
AFTER NOMINAL SUBJECT BASED SCORING
[10, 11, 12]
{}
[{1: 0, 4: 1}, {2: 2}, {7: 2, 14: 1}]


AFTER COMP CLAUSE BASED SCORING
[10, 11, 12]
{}
[{1: 0, 4: 1}, {2: 2}, {7: 2, 14: 1}]


AFTER AUX BASED SCORING
[10, 11, 12]
{}
[{1: 0, 4: 1}, {2: 2}, {7: 2, 14: 1}]


AFTER BACKTRACKING BASED SCORING
[10, 11, 12]
{}
[{1: 0, 4: 1}, {2: 2}, {7: 2, 14: 1}]


AFTER ADDING SCORE MULTIPLIERS
[10, 11, 12]
{}
[{1: 0.0, 4: 0.65}, {2: 1.6}, {7: 2, 14: 1}]
-----------------------------


BEFORE VERB LIST FILTERING
[20, 21, 22]
{2: 0, 7: 0}
[{0: 0}, {}, {1: 0}]


AFTER VERB LIST FILTERI

Entry - 44

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[2, 3, 4]
{1: 0, 3: 0}
[{1: 0}, {2: 0, 4: 0}, {2: 0, 3: 0, 5: 0, 6: 0}]


AFTER VERB LIST FILTERING
[2, 3, 4]
{}
[{}, {}, {3: 0}]


Prev 4 - 3 SAME SUBJECT TP + 2
AFTER NOMINAL SUBJECT BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 2}]


AFTER COMP CLAUSE BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 2}]


AFTER AUX BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 3}]


AFTER BACKTRACKING BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 3}]


AFTER ADDING SCORE MULTIPLIERS
[2, 3, 4]
{}
[{}, {}, {3: 3}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[2, 3, 4]
{1: 0, 3: 0}
[{1: 0}, {2: 0, 4: 0}, {2: 0, 3: 0, 5: 0, 6: 0}]


AFTER VERB LIST FILTERING
[2, 3, 4]
{}
[{}, {}, {3: 0}]


Prev 4 - 3 SAME SUBJECT TP + 2
AFTER NOMINAL SUBJECT BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 2}]


AFTER COMP CLAUSE BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 2}]


AFTER AUX BASED SCORING
[2, 3, 4]
{}
[{}, {}, {3: 3}]


AFTER BACKTRACKING BASED SCORING
[2

Entry - 47

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0, 1]
{4: 0, 8: 0, 9: 0}
[{1: 0}, {1: 0, 7: 0}]


AFTER VERB LIST FILTERING
[0, 1]
{}
[{}, {7: 0}]


Prev 1 - 7 SAME SUBJECT TP + 2
AFTER NOMINAL SUBJECT BASED SCORING
[0, 1]
{}
[{}, {7: 2}]


AFTER COMP CLAUSE BASED SCORING
[0, 1]
{}
[{}, {7: 2}]


AFTER AUX BASED SCORING
[0, 1]
{}
[{}, {7: 2}]


AFTER BACKTRACKING BASED SCORING
[0, 1]
{}
[{}, {7: 2}]


AFTER ADDING SCORE MULTIPLIERS
[0, 1]
{}
[{}, {7: 2}]
-----------------------------


BEFORE VERB LIST FILTERING
[15, 16, 17]
{1: 0}
[{1: 0}, {5: 0}, {1: 0, 8: 0}]


AFTER VERB LIST FILTERING
[15, 16, 17]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[15, 16, 17]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[15, 16, 17]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[15, 16, 17]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[15, 16, 17]
{}
[{}, {}, {}]


AFTER ADDING SCORE MULTIPLIERS
[15, 16, 17]
{}
[{}, {}, {}]
-----------------------------


Scoring in SPACY



In [3]:
for index, conversation in enumerate(text):
    print(str(index) + ") ", end = "\n")
    cleaned_text = re.sub("__eou__", "", text[index])
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = cleaned_text.strip()
    print(cleaned_text, end = "\n\n")
    resolved_conversation, resolved_conversation_parsed = append_resolved_vp(conversation_final_vpe_list[index], clean(text[index]))
    print(resolved_conversation, end = "\n\n=================================\n\n")

0) 
It is a fine day today , right ? Yes , it is. It ' s wonderful after the downpour . How nice seeing the sun back again ! The air is so fresh . I hope it stays nice like this for a long period of time . Well , the weather is so unpredictable this time of year . You're right.We don't know what is going to be like tomorrow . Look ! It looks as if it is going to rain . The weatherman predicts a fine day today . The weather is often quite different from the weather forecast .

It is a fine day today , right ? Yes , it is a fine day . It's wonderful after the downpour . How nice seeing the sun back again ! The air is so fresh . I hope it stays nice like this for a long period of time . Well , the weather is so unpredictable this time of year . You're right.We don't know what is going to be like tomorrow . Look ! It looks as if it is going to rain . The weatherman predicts a fine day today . The weather is often quite different from the weather forecast .


1) 
Would you like to go to a c

Where were you last night , at exactly nine twenty PM ? nine twenty pm , I couldn't say . I can't remember . Are you sure you can't remember ? I've absolutely certain . Were you in or out ? Can you remember that ? I'm not quite positive . I think I went out at about half pass nine . Did you see anyone come into the block flats ? No , I didn't see . Sure ? Quite sure .


15) 
So , Fred , what are your plans for after graduation ? Well , I've already got a job waiting for me back in my hometown . That's cool . Have you already found an apartment to live in ? I ’ m planning on living with my parents . Won't you ? I couldn't even if I wanted to . My parents told me that if I went home , then I'd have to find my own place . You mean they're kicking you out ? Not really , they just don't want me living at home . My older sister did that , and she lived at home for seven years . Once she started living at home , it got harder and harder for her to move out . Well , it's not like my parents wa

Thanks for inviting me to your friend's wedding . I was surprised to hear the bride and groom talking to each other that way during the marriage ceremony . Is that common ? Well , some people use a standard set of words , but many couples today write their own wedding vows . Your tradition of throwing rice as they got into their car was interesting , but it didn't look like real rice to me . It wasn't interesting . It was birdseed . We don't throw rice anymore because someone discovered that birds come along and eat the rice and have trouble digesting it . Oh , I didn't think of that . Well , this reception reminds me of weddingparties back in China . Plenty of food and drink , music and dancing , and interesting traditions . Wait until you see what happens next ! The bride is getting ready to throw her bouquet . Let's go over and watch . She's got quite a crowd around her . Oh , look ! That little girl caught it ! She's my friend's ten-year-old niece . I guess there won't be another w

Can you speak Japanese ? Yes , I can speak Japanese , but not very well . Where did you learn it ? At school . And I'm still learning it now . How long have you been learning it ? About three years . Can you speak any other languages ? I'm afraid not .


33) 
Did you go to the concert last weekend ? No , I didn't . And you ? Was it good ? Yes , I did . I enjoyed it a lot.There was a folk singer , a violinist and a pianist . How much was the ticket ? I only paid two dollars for mine .

Did you go to the concert last weekend ? No , I didn't go to the concert . And you ? Was it good ? Yes , I did . I enjoyed it a lot.There was a folk singer , a violinist and a pianist . How much was the ticket ? I only paid two dollars for mine .


34) 
Frank is getting married , do you believe this ? Is he really ? Yes , he is . He loves the girl very much . Who is he marring ? A girl he met on holiday in Spain , I think . Have they set a date for the wedding ? Not yet .

Frank is getting married , do yo

Hello , let me introduce myself . I'm your neighbor . My name is Shirley . Hi , Shirley . I'm Sophie . It's nice to meet you . Nice to meet you , too . Please come in . Here is a little gift for your family . It's a calendar with all the community events on it . I hope it can be helpful . I really appreciate it . Since we are neighbors , if you need any help or anything , please come to me . I will need any help . Actually , I am worried so much about the security of this community because my house has been burgled twice before . Oh , don't worry about that here . There are security guards on duty 24 hours a day and they patrol all over the neighborhood . You know , they are all very strong . Really ? That's good . Is there any supermarket in our neighborhood ? Of course . There is a Carrefour near the neighborhood . I 'll show you there if you want to show you . That's terrific ! By the way , a party will be held for new neighbors the day after tomorrow . I hope I can see you there . 